In [3]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import requests
import re

#Setup WebDriver with webdriver-manager
service = EdgeService(EdgeChromiumDriverManager().install())
options = EdgeOptions()
driver = webdriver.Edge(service=service, options=options)

In [4]:
URL = []
for i in range(1,4):
    service = EdgeService(EdgeChromiumDriverManager().install())
    options = EdgeOptions()
    driver = webdriver.Edge(service=service, options=options)
    url = f"https://www.imdb.com/list/ls596927598/?page={i}"
    driver.get(url)
    print(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    for i in soup.find_all("div", class_="ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-143a3ae8-2 ezwMlU dli-title with-margin"):
        a_tag = i.find("a")
        if a_tag and a_tag.get("href"):
            full_url = f"https://www.imdb.com{a_tag.get('href')}"
            URL.append(full_url)
    # Close the browser
    driver.quit()


https://www.imdb.com/list/ls596927598/?page=1
https://www.imdb.com/list/ls596927598/?page=2
https://www.imdb.com/list/ls596927598/?page=3


In [5]:
Title = [] 
Year=[]  
Rating = [] 
Directors = [] 
stars_list = []
Votes = []
Languages = []
Duration  = []
Gross = []
production_names = []
Genres = []


for i in URL:
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    response=requests.get(i,headers=request_header)
    print(response)
    soup=BeautifulSoup(response.text)
    
    for j in soup.find_all("span", class_ = 'hero__primary-text'):
        title=j.text.strip()
        if title:
            Title.append(title)
        else:
            Title.append(np.nan)

    year_found = False
    for k in soup.find_all("a", class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color'):
        text = k.text
        year = re.findall(r"^\d{4}", text)
        if year:
            Year.append(year[0])
            year_found = True
            break
    if not year_found:
        Year.append(np.nan)

    z_block = soup.find('div', class_ = 'sc-3b24bae4-11 iDIjTq')
    if z_block:
        text = z_block.text

        rat = re.findall(r"(\d\.\d)/10",text)   
        Rating.append(rat[0] if rat else np.nan)

        director = re.findall(r"Director[s]?(.*?)(Writer[s]?)", text)
        if director:
            director_names = director[0][0].strip()
            Directors.append(director_names)
        else:
            Directors.append(np.nan)

        votes = re.findall(r"10([\d.,KkMm]+)YOUR", text)   
        Votes.append(votes[0] if votes else np.nan)

        match = re.search(r"Stars(.*?)(?=\d+User reviews)", text)
        if match:
            raw_names = match.group(1).strip()
            names = re.findall(r'[A-Z][a-z]+(?:\s[A-Z][a-z]+)+', raw_names)
            stars_list.append(','.join(names))
        else:
            stars_list.append(np.nan)
    else:
        Rating.append(np.nan)
        Directors.append(np.nan)
        Votes.append(np.nan)
        stars_list.append(np.nan)

            

    d = soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-103e4e3c-2 cMcwpt baseAlt baseAlt')
    if d:
        match = re.findall(r"\d{1}h\s\d{1,2}m", d.text.strip())
        Duration.append(match[0] if match else np.nan)
    else:
        Duration.append(np.nan)

        

    genre_div = soup.find('div', class_='ipc-chip-list--baseAlt ipc-chip-list ipc-chip-list--nowrap sc-865706aa-4 bjiBzC')
    if genre_div:
        genre_tags = genre_div.find_all('a')
        genre_names = [g.text.strip() for g in genre_tags]
        Genres.append(', '.join(genre_names))
    else:
        Genres.append(np.nan)

    language_section = soup.find('li', attrs={'data-testid': 'title-details-languages'})

    if language_section:
        langs = [lang.text.strip() for lang in language_section.find_all('a')]
        Languages.append(", ".join(langs))  
    else:
        Languages.append(np.nan)

    
    gross_found = False
    for li in soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list__item--align-end sc-a3d75170-2 eyloUU'):
        text = li.text.strip()
        match = re.search(r'Gross worldwide\$(\d[\d,]*)', text)
        if match:
            Gross.append(match.group(1))
            gross_found = True
            break
    if not gross_found:
        Gross.append(np.nan)

    prod_section = soup.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-all ipc-metadata-list--base')
    if prod_section:
        text = prod_section.text.strip()
        match = re.findall(r'Production compan(?:y|ies)(.*?)(?=See)', text)
        if match:
            cleaned = re.sub(r'(?<=[a-z])(?=[A-Z])', ', ', match[0].strip())
            production_names.append(cleaned)
        else:
            production_names.append(np.nan)
    else:
        production_names.append(np.nan) 



df = pd.DataFrame({
    'Title': Title,
    'Year': Year,
    'Rating': Rating,
    'Directors': Directors,
    'Stars': stars_list,
    'Votes': Votes,
    'Languages': Languages,
    'Duration': Duration,
    'Gross Worldwide': Gross,
    'Production Company': production_names,
    'Genres': Genres
})

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [11]:
print([len(lst) for lst in [Title, Year, Rating, Directors, stars_list, Votes, Languages, Duration, Gross, production_names, Genres]])

[698, 698, 698, 698, 698, 698, 698, 698, 698, 698, 698]


In [14]:
df

,Title,Year,Rating,Directors,Stars,Votes,Languages,Duration,Gross Worldwide,Production Company,Genres
0,Jaat,2025,6.7,Gopichand Malineni,"Sunny Deol,Randeep Hooda,Regena Cassandrra",5.6K,Hindi,2h 38m,"531,499","Mythri Movie Makers, People Media Factory","Action Epic, One-Person Army Action, Action, D..."
1,Chhaava,2025,7.4,Laxman Utekar,"Vicky Kaushal,Akshaye Khanna,Rashmika Mandanna",31K,Hindi,2h 41m,"8,219,603",Maddock Films,"Action Epic, Costume Drama, Epic, Historical E..."
2,Good Bad Ugly,2025,7.0,Adhik Ravichandran,"Trisha Krishnan,Ajith Kumar,Priya Prakash Varrier",8.2K,Tamil,2h 19m,"1,462,902",Mythri Movie Makers,"Action, Drama, Thriller"
3,Court - State vs a Nobody,2025,7.9,Ram Jagadeesh,"Priyadarshi Pulikonda,Harsh Roshan,Sri Devi",5.7K,Telugu,2h 35m,"133,791",Wall Poster Cinema,"Legal Drama, Drama"
4,Deva,2025,6.8,Rosshan Andrrews,"Shahid Kapoor,Pooja Hegde,Saadhika Syal",24K,Hindi,2h 36m,"1,091,296","Roy Kapur Films, Zee Studios","Cop Drama, One-Person Army Action, Police Proc..."
...,...,...,...,...,...,...,...,...,...,...,...
693,Haider,2014,8.0,Vishal Bhardwaj,"Shahid Kapoor,Shraddha Kapoor",59K,Hindi,2h 40m,"1,404,307","UTV Motion Pictures, Vishal Bhardwaj Pictures","Action, Crime, Drama, Thriller"
694,Shanmukha,2025,3.4,Shanumugam Sappani,"Avika Gor,Krishna Alluri",358,Telugu,2h 1m,NaN,Sapbro Productions,"Fantasy, Thriller"
695,Liger,2022,2.6,Puri Jagannadh,"Vijay Deverakonda,Ananya Panday,Ramya Krishnan",48K,"Hindi, Telugu",2h 20m,"4,879,486","Dharma Productions, Puri Connects","Martial Arts, Action, Drama, Romance, Sport"
696,Demonte Colony 2 (Vengeance of the Unholy),2024,6.6,R. Ajay Gnanamuthu,,4.4K,"Tamil, Telugu",2h 24m,"142,987","BTG Universal, Gnanamuthu Pattarai, Raaj Varma...","Horror, Thriller"


In [16]:
df['Languages'].dropna().str.split(', ').value_counts()

Languages
[Malayalam]                                   124
[Kannada]                                     123
[Telugu]                                      123
[Tamil]                                       122
[Hindi]                                       115
[Hindi, English]                               21
[Malayalam, Tamil]                              5
[Telugu, Tamil]                                 5
[Tamil, Hindi]                                  4
[Hindi, Urdu]                                   3
[Malayalam, Kannada]                            3
[Tamil, Telugu]                                 3
[Malayalam, Telugu]                             3
[Bengali]                                       2
[Kannada, Tamil]                                2
[Kannada, Telugu, Tamil]                        2
[Kannada, Telugu, Tamil, Malayalam]             2
[Tamil, Telugu, Kannada, Malayalam]             2
[Malayalam, English, Arabic]                    1
[Kannada, Telugu]                       

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Title               698 non-null    object
 1   Year                698 non-null    object
 2   Rating              658 non-null    object
 3   Directors           647 non-null    object
 4   Stars               620 non-null    object
 5   Votes               658 non-null    object
 6   Languages           698 non-null    object
 7   Duration            689 non-null    object
 8   Gross Worldwide     457 non-null    object
 9   Production Company  690 non-null    object
 10  Genres              698 non-null    object
dtypes: object(11)
memory usage: 60.1+ KB


In [67]:
df.to_csv('IMDB01.csv',index=False)